In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [2]:
# Set up the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
# Hyperparameters
sequence_length = 28
input_size = 28
hidden_size = 128
num_layers = 2
num_classes = 10
batch_size = 100
num_epochs = 2
learning_rate = 0.003

In [6]:
# MNIST dataset
train_dataset = torchvision.datasets.MNIST(root='./data',
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='./data',
                                          train=False, 
                                          transform=transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)

In [7]:
class BiRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(BiRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first = True, bidirectional = True)
        self.fc = nn.Linear(hidden_size * 2, num_classes)
    
    def forward(self, x):
        h0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size).to(device)
        
        out, _ = self.lstm(x, (h0, c0))   # out -> Tensor of shape (batch_size, seq_length, hidden_size * 2)
        out = self.fc(out[:, -1, :])
        
        return out

In [8]:
model = BiRNN(input_size, hidden_size, num_layers, num_classes).to(device)

In [9]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [11]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        _, argmax = torch.max(outputs, 1)
        accuracy = (labels == argmax.squeeze()).float().mean()

        if (i + 1) % 100 == 0:
            print(f'Epoch: {epoch + 1}, Step: [{i + 1}/{total_step}], Accuracy: {accuracy}, Loss: {loss.item()}')

Epoch: 1, Step: [100/600], Accuracy: 0.7099999785423279, Loss: 0.9242953658103943
Epoch: 1, Step: [200/600], Accuracy: 0.8899999856948853, Loss: 0.2969509959220886
Epoch: 1, Step: [300/600], Accuracy: 0.9399999976158142, Loss: 0.25136739015579224
Epoch: 1, Step: [400/600], Accuracy: 0.9200000166893005, Loss: 0.3290531039237976
Epoch: 1, Step: [500/600], Accuracy: 0.9700000286102295, Loss: 0.12089239805936813
Epoch: 1, Step: [600/600], Accuracy: 0.9599999785423279, Loss: 0.12319962680339813
Epoch: 2, Step: [100/600], Accuracy: 0.8999999761581421, Loss: 0.30286750197410583
Epoch: 2, Step: [200/600], Accuracy: 1.0, Loss: 0.01562455017119646
Epoch: 2, Step: [300/600], Accuracy: 0.9599999785423279, Loss: 0.10631554573774338
Epoch: 2, Step: [400/600], Accuracy: 0.9800000190734863, Loss: 0.05908649414777756
Epoch: 2, Step: [500/600], Accuracy: 0.9800000190734863, Loss: 0.049212049692869186
Epoch: 2, Step: [600/600], Accuracy: 0.9800000190734863, Loss: 0.08120422065258026


In [13]:
# Test the model
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print(f'Accuracy of the model: {correct * 100 / total}')

Accuracy of the model: 97.93


In [14]:
# Save the model
PATH = './mnist_bidirectional_rnn.pth'
torch.save(model.state_dict(), PATH)